# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации


### Описание данных: 
<br> 1. children — количество детей в семье
<br> 2. days_employed — общий трудовой стаж в днях
<br> 3. dob_years — возраст клиента в годах
<br> 4. education — уровень образования клиента
<br> 5. education_id — идентификатор уровня образования
<br> 6. family_status — семейное положение
<br> 7. family_status_id — идентификатор семейного положения
<br> 8. gender — пол клиента
<br> 9. income_type — тип занятости
<br> 10. debt — имел ли задолженность по возврату кредитов
<br> 11. total_income — ежемесячный доход
<br> 12. purpose — цель получения кредита

### Подключение библиотек:

In [2]:
import pandas as pd
import math
from IPython.display import display
from pymystem3 import Mystem
from collections import Counter

### Изучение общей информации:

In [2]:
# Загрузим датастет и взглянем на его первые 10 строк
df = pd.read_csv('/datasets/data.csv')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Уже можно заметить следующие аномалии:
1. Отрицательный стаж работы
2. Разный регистр в столбце 'education'
3. Формат данных в столбцах 'days_employed' и 'total_income' float64 (нужно исправить на int)

In [3]:
# Взглянем на общую информацию о таблице:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что есть пропуски в столбцах 'days_employed' и 'total_income'. 
В дальнейшем нужно будет проверить, находятся ли они в одних и тех же строках.
А пока что взглянем на основные стат. данные по нашему датасету:

In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод:**

Видим следующие аномалии:
1. Минимальное значение в столбце 'children' = -1
2. Максимальное значение в столбце 'children' = 20
3. Среднее значение стажа работы, переведенное в года = 172 года
4. Отрицательные значения стажа работы
5. Минимальное значение возраста = 0 лет
6. Нечитабельные значения 'total_income' из-за формата float

##  Предобработка данных

### Обработка пропусков

Начнем с проверки гипотезы: действительно ли пропуски в days_employed и total_income находятся в одной строке:

In [5]:
data_gaps = df.loc[(df['days_employed'].isna() == True) & (df['total_income'].isna() == True)]

print(data_gaps.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB
None


<br> Действительно пропуски в days_employed и total_income находятся в одних и тех же строках.
<br> Для заполнения данных пропусков, необходимо будет в первую очередь заменить отрицательные значения на положительные, и после этого рассчитать среднее значение для days_employed и медиану для total_income.

<br> Для заполнения данных пропусков, будем действовать по следующему алгоритму:
<br>  1. Заменим отрицательные значения на положительные.
<br>  2. Отбросим невозможные значения стажа работы, за максимально возможное значение будем использовать "более 60 лет" (например, человек работает с 14 лет, с выходом на пенсию в 70 лет, ± 4 года на различные жизненные обстоятельства).
<br>  3. После этого напишем функцию, которая установит числовой идентификатор (1 - подходит, 0 - не подходит), опираясь на пороговое значение.

In [6]:
# Заменим отрицательные значения в days_employed на положительные:

def negative_to_positive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
    
df['days_employed'] = df['days_employed'].apply(negative_to_positive)
 
# Проверим правильность выполнения  
display(df[df['days_employed'] < 0].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
children            0 non-null int64
days_employed       0 non-null float64
dob_years           0 non-null int64
education           0 non-null object
education_id        0 non-null int64
family_status       0 non-null object
family_status_id    0 non-null int64
gender              0 non-null object
income_type         0 non-null object
debt                0 non-null int64
total_income        0 non-null float64
purpose             0 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 0.0+ bytes


None

Отрицательные числа устранены, теперь можно переходить к расстановке идентификаторов для стажа работы:

In [7]:
def years_employed_check(days_employed):
    try:
        if days_employed == None or math.isnan(days_employed):
            return 1
        years_employed = math.floor(days_employed / 365)
    except:
        return 0
    if years_employed <= 60:
        return 1
    else:
        return 0

# Применим функцию, и проверим правильность выполнения
df['years_employed_check'] = df['days_employed'].apply(years_employed_check)
display(df.head(5))
display(df['years_employed_check'].isnull().value_counts())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed_check
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,1
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0


False    21525
Name: years_employed_check, dtype: int64

Вспомогательный столбик готов, приступим к заполнению пропусков:

In [8]:
#Отфильтруем таблицу с репрезентативными значениями стажа работы

df_years_employed_check = df[df['years_employed_check'] == 1]
print(df_years_employed_check.info())

# Теперь взглянем на таблицу со средним стажем, сгруппированную по возрасту клиента
mean_days_by_years = df_years_employed_check.groupby('dob_years')['days_employed'].aggregate(['mean'])
display(mean_days_by_years)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18080 entries, 0 to 21524
Data columns (total 13 columns):
children                18080 non-null int64
days_employed           15906 non-null float64
dob_years               18080 non-null int64
education               18080 non-null object
education_id            18080 non-null int64
family_status           18080 non-null object
family_status_id        18080 non-null int64
gender                  18080 non-null object
income_type             18080 non-null object
debt                    18080 non-null int64
total_income            15906 non-null float64
purpose                 18080 non-null object
years_employed_check    18080 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 1.9+ MB
None


,mean
dob_years,
0,2200.375775
19,633.678086
20,684.944308
21,709.440930
22,781.376775
23,827.309437
24,1026.405485
25,1088.406453
26,1200.288052


<br> Несмотря на то что у пенсионеров 71, 73 и 75 лет средний стаж работы отклоняется от общей тенденции,
<br> в целом такие значения помогут составить более объективную картину после заполнения пропусков.


In [9]:
# Теперь сгруппируем данные по возрасту клиента, 
# получим среднее значение стажа работы в зависимости от возраста, 
# и заполним данными значениями пропуски в соответствии с возрастом клиента
df['days_employed'] = df['days_employed'].fillna(df_years_employed_check.groupby('dob_years')['days_employed'].transform('mean'))

# Проверим правильность выполнения
display(df.info())
display(df['days_employed'].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children                21525 non-null int64
days_employed           21525 non-null float64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            19351 non-null float64
purpose                 21525 non-null object
years_employed_check    21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


None

count     21525.000000
mean      60418.896545
std      133240.056264
min          24.141633
25%        1013.920085
50%        2307.062965
75%        4781.798667
max      401755.400475
Name: days_employed, dtype: float64

Пропуски в 'days_employed' устранены, теперь перейдем к замене нулевых значений в столбце 'dob_years'

In [10]:
# нулевое значение возраста будем заполнять средним значением по выборке
mean_dob_years = df['dob_years'].mean()
df['dob_years'] = df['dob_years'].replace(0, mean_dob_years)

# Проверим правильность выполнения
display(df['dob_years'].describe())


count    21525.000000
mean        43.496522
std         12.218174
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


Теперь аналогично заполним пропуски в уровене дохода. За определяющие факторы попробуем использовать уровень образования и тип занятости. Так как, в основном, именно эти два фактора определяют уровень дохода у человека


In [11]:
# Так как для заполнения пропусков нам необходимы всего три показателя, 
# для удобства создадим отдельный срез таблицы

df_education_inctype = df.loc[:, ['education_id', 'income_type', 'total_income']]

#Сгруппируем по двум факторам, посчитаем среднее значение, и проверим правильность 
median_income_by_education_inctype = df_education_inctype.loc[:, ['education_id', 'income_type', 'total_income']].groupby(['education_id', 'income_type']).aggregate(['median'])
display(median_income_by_education_inctype)

total_income
                                     median
education_id income_type                   
0            безработный      202722.511368
             госслужащий      172511.107016
             компаньон        201785.400018
             пенсионер        144240.768611
             предприниматель  499163.144947
             сотрудник        165640.744634
             студент           98201.625314
1            безработный       59956.991984
             в декрете         53829.130729
             госслужащий      136652.970357
             компаньон        159070.690289
             пенсионер        114842.854099
             сотрудник        136555.108821
2            госслужащий      160592.345303
             компаньон        179867.152890
             пенсионер        120136.896353
             сотрудник        151308.937846
3            госслужащий      148339.290825
             компаньон        136798.905143
             пенсионер        102598.653164
             сотрудник        125994.910603
4            госслужащий      111392.231107
             пенсионер        177088.845999
             сотрудник        198570.757322

Видим, что результаты не отображают объективную картину, так как у людей со средним образованием почему-то 
уровень дохода отличается от групп с высшим образованием, особенно у госслужащих и пенсионеров.

In [12]:
# Тогда сгруппируем по одному фактору - типу занятости, и проверим правильность
median_income_by_education_inctype = df_education_inctype.loc[:, ['income_type', 'total_income']].groupby(['income_type']).aggregate(['median'])
display(median_income_by_education_inctype)

,total_income
,median
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Такие значения больше похожи на правду, и можно заметить что студенты, и люди в декрете
получают наименьший доход в нашем датасете.


In [13]:
# Заполняем пропуски полученными результатами, и проверим правильность выполнения
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('mean'))
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children                21525 non-null int64
days_employed           21525 non-null float64
dob_years               21525 non-null float64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null float64
purpose                 21525 non-null object
years_employed_check    21525 non-null int64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB
None


<br> Теперь, когда все пропуски заполнены, переходим к исправлению аномалий в количестве детей

<br> Для начала узнаем сколько строк с количеством детей = 20 и = -1 присутствует
<br> А также взглянем на уникальные значения в этом столбце

In [14]:
display(df['children'].value_counts())
print('Количество детей = 20, строк - ', df[df['children'] == 20].count()[0])
print('Количество детей = -1, строк - ', df[df['children'] == -1].count()[0])

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей = 20, строк -  76
Количество детей = -1, строк -  47


Видим, что аномалии не слишком распространены, поэтому будем принимать их значения 20 и -1 за опечатку, 
и заменим их на 2 и 1, соответственно,
так как репрезентативность выборки от этого практически не пострадает.

In [15]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

#Проверим правильность
display(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Также приведем к общему виду столбец "education", так как там наблюдается разный регистр букв

In [16]:
# Для начала проверим, не упускаем ли мы что-то помимо регистра букв
display((df['education'].value_counts()))

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Различия наблюдаются только в регистре, поэтому спокойно приводим все значения к нижнему регистру

In [17]:
df['education'] = df['education'].str.lower()

#Проверим правильность
display(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Все пропуски и аномалии устранены, теперь переходим к замене типа данных в 'days_employed' и 'total_income'

### Замена типа данных

Для замены типа данных в обоих столбцах будем использовать astype() так как данные в этих столбцах чисты и имеют тип float64

In [18]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

# Проверим:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children                21525 non-null int64
days_employed           21525 non-null int64
dob_years               21525 non-null float64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null int64
purpose                 21525 non-null object
years_employed_check    21525 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


None

Теперь можем приступать к обработке дубликатов

### Обработка дубликатов

Для начала выясним, сколько всего потенциальных дубликатов имеется в нашем датасете:

In [19]:
print(df.duplicated().sum())

71


Теперь взглянем на них:

In [20]:
display(df[df.duplicated(keep = False)].sort_values(by = ['days_employed', 'purpose'], ascending = False))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed_check
13035,0,4059,65.0,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу,1
20187,0,4059,65.0,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу,1
6674,0,4045,64.0,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу,1
17338,0,4045,64.0,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу,1
7313,0,4045,64.0,высшее,0,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18328,0,1553,29.0,высшее,0,женат / замужем,0,M,сотрудник,0,161380,покупка жилой недвижимости,1
8853,1,827,23.0,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу,1
20297,1,827,23.0,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу,1
15892,0,827,23.0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161380,сделка с подержанным автомобилем,1


Выглядят действительно как дубликаты, вполне вероятно что они возникли из-за человеческого фактора (по аналогии с 'children' = 20)
<br> Необходимо избавиться от них:

In [21]:
df = df.drop_duplicates()

#Проверим
print(df.duplicated().sum())

0


Теперь, когда датасет вычищен от пропусков, аномальных значений, и пропусков, переходим к лемматизации цели кредита

### Лемматизация


In [22]:
# Для начала взглянем на уникальные значения в столбце 'purpose'
unique_purpose = df['purpose'].unique()
display(unique_purpose)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [23]:
# Используя этот список, начнем лемматизацию с помощью pymystem3
m = Mystem()
lemmatize_list = []
for purpose in unique_purpose:
    lemmatize_word = m.lemmatize(purpose)
    lemmatize_list.extend(lemmatize_word)

unique_lemmas = Counter(lemmatize_list)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse = True)

[(' ', 59),
 ('\n', 38),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('подержать', 1),
 ('со', 1),
 ('подержанный', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1)]

In [24]:
# Избавимся от предлогов, частиц, глаголов в инфинитиве и прилагательных:
cleared_lemmatize_list = []
for i in lemmatize_list:
    if len(i) > 3 and i != ' ' and i != '\n' and i.find('ть') == -1 and i.find('й', len(i)-1, len(i)) == -1:
        cleared_lemmatize_list.append(i)

cleared_unique_lemmas = Counter(cleared_lemmatize_list)
sorted(cleared_unique_lemmas.items(), key = lambda pair: pair[1], reverse = True)

[('покупка', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('операция', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('сделка', 2),
 ('приобретение', 1),
 ('проведение', 1),
 ('семья', 1),
 ('сдача', 1),
 ('ремонт', 1)]

Используя данный список, вполне можно начать проведение категоризации данных:

### Категоризация данных

Начнем с категоризации цели кредита, используя список полученный в прошлом пункте:
Разделим его на четыре категории:
    1. 'Покупка жилья'
    2. 'Покупка автомобиля'
    3. 'Получение образования'
    4. 'Проведение свадьбы'
Так как видно, что: 
    1. Леммы 'недвижимость', 'операция', 'жилье', 'строительство', 'ремонт', 'сдача', 'семья' - относятся к покупке жилья
    2. Везде, где есть лемма 'автомобиль', 'сделка' - связаны с покупкой автомобиля
    3. 'Образование', 'получение' связано с получением образованием
    4. И 'проведение', 'свадьба' - связаны с проведением свадьбы


In [25]:
realty = ['покуп', 'жиль', 'строи', 'недвиж']
car = ['автомоб']
education = ['образов']
marriage = ['свадьб']


# Создадим функцию для внесения категории в новый столбец датасета
df['purpose_category'] = 0
def set_purpose_category(category_list, category):
    join = '|'.join(category_list)
    index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
    for i in index:
        df.loc[i, 'purpose_category'] = category
    return df

# Присвоим категории через созданную функцию
set_purpose_category(realty, 'Покупка жилья') 
set_purpose_category(car, 'Покупка автомобиля') 
set_purpose_category(education, 'Получение образования') 
set_purpose_category(marriage, 'Проведение свадьбы') 

# Проверим не осталось ли пустых значений в новом столбце
print(df['purpose_category'].value_counts())
print(df.loc[(df['purpose_category'].isnull() == True)])

Покупка жилья            10811
Покупка автомобиля        4306
Получение образования     4013
Проведение свадьбы        2324
Name: purpose_category, dtype: int64
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, years_employed_check, purpose_category]
Index: []


Категоризация по цели кредита завершена, теперь организуем создание категорий для других показателей, начнем со стажа:
Используя стаж работы в днях, преобразуем его в года, округляя вниз, и категоризируем:   

In [26]:
def days_employed_categorize(days_employed):
    years_employed = math.floor(days_employed / 365)
    if years_employed <= 5:
        return 'до 5 лет'
    elif years_employed > 5 and years_employed <= 10:
        return 'от 5 до 10 лет'
    elif years_employed > 10 and years_employed <= 20:
        return 'от 10 до 20 лет'
    else:
        return 'свыше 20 лет'

# Применим функцию, и проверим её работоспособность
df['days_employed_category'] = df['days_employed'].apply(days_employed_categorize)
display(df['days_employed_category'].value_counts())

до 5 лет           10346
от 5 до 10 лет      4838
свыше 20 лет        4112
от 10 до 20 лет     2158
Name: days_employed_category, dtype: int64

<br> Теперь приступить к категоризации датасета по уровню дохода:
<br> Категоризировать будем следующим образом:
<br>    1. Низший класс
<br>    2. Средний класс
<br>    3. Высший класс
<br> Так как нам неизвестно кредитный отдел какого банка, какого города, и какой страны выслал этот датасет, строить предположения на догадках будет неверно. 
Поэтому, сначала взглянем на общую стат. информацию по столбцу 'total_income', чтобы определиться с интервалом, по которому можно будет определить средний класс.

In [27]:
display(df['total_income'].describe().astype(int))

count      21454
mean      167431
std        98060
min        20667
25%       107623
50%       151887
75%       202417
max      2265604
Name: total_income, dtype: int64

Видим что медиана (50%) равна 150 тыс., 25% от датасета получают достаточно крупную сумму в месяц - 100 тыс., и 75% от датасета получают 200 тыс. в месяц.
Поэтому основным показателем для установления интервала будем использовать медиану, сам интервал установим в ± 30 тысяч от медианы, так как он разобъет нашу выборку на три приблизительно равные части.

In [28]:
medium_income = df['total_income'].median()
floor_medium_income = medium_income - 30000
ceil_medium_income = medium_income + 30000

def income_categorize(total_income):
    if total_income < floor_medium_income:
        return 'Низший класс'
    elif floor_medium_income < total_income and total_income < ceil_medium_income:
        return 'Средний класс'
    elif total_income > ceil_medium_income:
        return 'Высший класс'
        
# Применим функцию, и проверим её работоспособность
df['income_category'] = df['total_income'].apply(income_categorize)
display(df['income_category'].value_counts())    

Средний класс    7618
Низший класс     7040
Высший класс     6796
Name: income_category, dtype: int64

И последним этапом в категоризации нашего датасета, будет категоризация по возрасту:

In [29]:
def dob_years_categorize(dob_years):
    if dob_years <= 30:
        return 'Молодежь'
    elif dob_years > 30 and dob_years < 55:
        return 'Средний возраст'
    elif dob_years >= 55 and dob_years < 65:
        return 'Предпенсионный возраст'
    elif dob_years >= 65:
        return 'Пенсионер'

# Применим функцию, и проверим её работоспособность
df['dob_years_category'] = df['dob_years'].apply(dob_years_categorize)
display(df['dob_years_category'].value_counts())

Средний возраст           12958
Предпенсионный возраст     3884
Молодежь                   3717
Пенсионер                   895
Name: dob_years_category, dtype: int64

И напоследок, сведем уже готовые категории по образованию, и типу занятости для наглядности:

In [30]:
#display(df['education'].value_counts())

При расчете зависимостей по образованию и возврату кредита в срок, следует обратить внимание, что клиентов с ученой степенью наименьшее количество.

In [31]:
#display(df['income_type'].value_counts())

По аналогии с предыдущим выводом, следует обратить внимание, что категории 'безработный', 'предприниматель' 'в декрете' и 'студент' практически не влияют на выборку.

Для типа образования и семейного статуса в датасете присутствуют числовые идентификаторы, что намекает нам на готовую категоризацию по этим показателям, проверим, так ли это:

In [32]:
display(df.groupby('education')['education_id'].unique())
display(df.groupby('family_status')['family_status_id'].unique())

education
высшее                 [0]
начальное              [3]
неоконченное высшее    [2]
среднее                [1]
ученая степень         [4]
Name: education_id, dtype: object

family_status
Не женат / не замужем    [4]
в разводе                [3]
вдовец / вдова           [2]
гражданский брак         [1]
женат / замужем          [0]
Name: family_status_id, dtype: object

<br> Видим, что каждому уникальному типу образованию и семейному статусу соответствует свой уникальный числовой идентификатор.
<br> Оставшуюся категоризацию по количеству детей выполнять не имеет практического смысла, так как количество детей само по себе уже является категорией. 
<br> Соответственно, категоризация завершена, теперь можно приступать к поиску зависимостей.

## Выявление зависимостей

<br> Для выявления зависимости, будем использовать простой метод через среднее значение показателя 'debt':
<br> Чем оно ближе к 1, тем меньше вероятность возврата кредита в срок
<br> Чем оно ближе к 0, тем больше вероятность возврата кредита в срок

<br> Используем для этого следующую функцию, которая на входе будет принимать столбец категории, 
<br> А на выходе выдавать сводную таблицу с количеством клиентов по категории, количеством должников, и вероятностью невозврата кредита в срок:

In [33]:
def correlation(category):
    corr_table = df.pivot_table(values = ['debt'], index = [category], aggfunc= {'debt': ['count', 'sum', 'mean']})
    сorr_table = corr_table.rename({'count': 'Кол-во клиентов', 'mean' : 'Вероятность невозврата в срок', 'sum' : 'Кол-во должников'}, axis = 'columns', inplace = True)
    return corr_table


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
display(correlation('children').sort_values(by = 'children', ascending = True))

debt                                               
         Кол-во клиентов Вероятность невозврата в срок Кол-во должников
children                                                               
0                14091.0                      0.075438           1063.0
1                 4855.0                      0.091658            445.0
2                 2128.0                      0.094925            202.0
3                  330.0                      0.081818             27.0
4                   41.0                      0.097561              4.0
5                    9.0                      0.000000              0.0

**Вывод:** Зависимость между наличием детей и возвратом кредита в срок присутствует, клиенты у которых дети отсутствуют, возвращают кредит в срок чаще, чем клиенты у которых от 1 до 4 детей. Однако, лучше всего исполняют свои обязательства по выплатам клиенты, у которых пятеро детей. Чаще всего не выплачивают кредит в срок клиенты с 4-мя детьми, клиенты же с одним или двумя детьми приблизительно одинаково плохо исполняют свои обязательства. Однако, выбирая между клиентом с 1 или 2 детьми, всё же лучше обратить внимание на клиента с 1 ребенком.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
display(correlation('family_status'))

debt                                \
                      Кол-во клиентов Вероятность невозврата в срок   
family_status                                                         
Не женат / не замужем          2810.0                      0.097509   
в разводе                      1195.0                      0.071130   
вдовец / вдова                  959.0                      0.065693   
гражданский брак               4151.0                      0.093471   
женат / замужем               12339.0                      0.075452   

                                        
                      Кол-во должников  
family_status                           
Не женат / не замужем            274.0  
в разводе                         85.0  
вдовец / вдова                    63.0  
гражданский брак                 388.0  
женат / замужем                  931.0

**Вывод:** Вдовцы/вдовы, клиенты в разводе, женатые/замужние клиенты возвращают кредит в срок примерно с одинаковой вероятностью. Клиенты, находящиеся в гражданском браке, или не женатые/не замужние чаще нарушают сроки выплаты по кредиту.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Вывод не противоречит полученному результату. 

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
display(correlation('income_category'))

debt                                               
                Кол-во клиентов Вероятность невозврата в срок Кол-во должников
income_category                                                               
Высший класс             6796.0                      0.073573            500.0
Низший класс             7040.0                      0.081392            573.0
Средний класс            7618.0                      0.087687            668.0

**Вывод:** Довольно очевидно, что клиенты с более высоким заработком возвращают кредит в срок. Однако, клиенты с низким заработком чаще соблюдают сроки выплаты по кредиту, чем клиенты со средним заработком.

- Как разные цели кредита влияют на его возврат в срок?

Проверим зависимость между стажем работы и возвратом кредита в срок:

In [37]:
display(correlation('days_employed_category'))

debt                                \
                       Кол-во клиентов Вероятность невозврата в срок   
days_employed_category                                                 
до 5 лет                       10346.0                      0.100425   
от 10 до 20 лет                 2158.0                      0.054680   
от 5 до 10 лет                  4838.0                      0.077511   
свыше 20 лет                    4112.0                      0.050827   

                                         
                       Кол-во должников  
days_employed_category                   
до 5 лет                         1039.0  
от 10 до 20 лет                   118.0  
от 5 до 10 лет                    375.0  
свыше 20 лет                      209.0

**Вывод:** Так же достаточно очевиден, чем больше стаж работы, тем клиент менее склонен к просрочкам по выплатам

Проверим зависимость между возрастом клиента и возвратом кредита в срок:

In [38]:
display(correlation('dob_years_category'))

debt                                \
                       Кол-во клиентов Вероятность невозврата в срок   
dob_years_category                                                     
Молодежь                        3717.0                      0.108421   
Пенсионер                        895.0                      0.054749   
Предпенсионный возраст          3884.0                      0.055098   
Средний возраст                12958.0                      0.082960   

                                         
                       Кол-во должников  
dob_years_category                       
Молодежь                          403.0  
Пенсионер                          49.0  
Предпенсионный возраст            214.0  
Средний возраст                  1075.0

**Вывод:** Аналогично с предыдущим выводом, чем старше клиент, тем реже он нарушает свои обязательства по выплатам

Проверим зависимость между целями кредита и возвратом кредита в срок:

In [39]:
display(correlation('purpose_category'))

debt                                \
                      Кол-во клиентов Вероятность невозврата в срок   
purpose_category                                                      
Покупка автомобиля             4306.0                      0.093590   
Покупка жилья                 10811.0                      0.072334   
Получение образования          4013.0                      0.092200   
Проведение свадьбы             2324.0                      0.080034   

                                        
                      Кол-во должников  
purpose_category                        
Покупка автомобиля               403.0  
Покупка жилья                    782.0  
Получение образования            370.0  
Проведение свадьбы               186.0

**Вывод:** Покупка жилья и проведение свадьбы для большинства людей это более серьезный и ответственный шаг, чем покупка автомобиля и получение образования. Поэтому такая зависимость вполне закономерна.

Проверим зависимость между образованием клиента и возвратом кредита в срок:

In [40]:
display(correlation('education'))

debt                                \
                    Кол-во клиентов Вероятность невозврата в срок   
education                                                           
высшее                       5250.0                      0.052952   
начальное                     282.0                      0.109929   
неоконченное высшее           744.0                      0.091398   
среднее                     15172.0                      0.089902   
ученая степень                  6.0                      0.000000   

                                      
                    Кол-во должников  
education                             
высшее                         278.0  
начальное                       31.0  
неоконченное высшее             68.0  
среднее                       1364.0  
ученая степень                   0.0

**Вывод:** Как уже было сказано в конце пункта категоризации данных, размер выборки клиентов с ученой степенью достаточно мал. Поэтому делать вывод о том, что все люди с ученой степенью, всегда выплачивают кредит в указанный срок - пока что рано. Однако, именно в этом датасете, так и произошло. 
В остальном, вывод тоже достаточно закономерен, чем выше качество образования у клиента - тем реже он пропускает установленные сроки выплаты по кредиту

И напоследок, проверим зависимость между типом занятости и возвратом кредита в срок. Для получения более корректных значений, будем использовать два фактора: категория доходов и тип занятости. 
Для этого вместо функции, создадим сводную таблицу, и применим к ней aggfunc для расчета зависимости.

In [41]:
correlation_pivot_table = df.pivot_table(values = ['debt'], index = ['income_category', 'income_type'], aggfunc= 'mean')
display(correlation_pivot_table)

debt
income_category income_type              
Высший класс    безработный      0.000000
                госслужащий      0.057778
                компаньон        0.065749
                пенсионер        0.060056
                предприниматель  0.000000
                сотрудник        0.086018
Низший класс    безработный      1.000000
                в декрете        1.000000
                госслужащий      0.069717
                компаньон        0.086142
                пенсионер        0.047513
                сотрудник        0.097286
                студент          0.000000
Средний класс   госслужащий      0.051095
                компаньон        0.080344
                пенсионер        0.066465
                сотрудник        0.101103

**Вывод:** По аналогии с предыдущим выводом, делать выводы о предпринимателях, студентах, безработных и людей в декрете - не имеет практического смысла. По оставшимся категориям, можно заметить что пенсионеры и госслужащие вдвое чаще выплачивают кредиты в срок, нежели простые наемные сотрудники. 

## Общий вывод

<br> **Используя полученные результаты, мы вполне можем составить картину *идеального кандидата* для получения кредита:**
<br>     · Дети: *более 2-х детей (либо отсутствуют)*
<br>     · Семейное положение: *состоит/состоял в официальном браке*
<br>     · Уровень доходов: *относится к высшему классу*
<br>     · Стаж работы: *более 10 лет*
<br>     · Возраст: *больше 55 лет*
<br>     · Цель кредита: *покупка жилья*
<br>     · Уровень образования: *высшее*
<br>     · Тип занятости: *на пенсии, либо на госслужбе*

<br> **Положительно-нейтральный кандидат выглядит следующим образом**:
<br>     · Дети: *более 2-х детей*
<br>     · Семейное положение: *в официальном или гражданском браке*
<br>     · Уровень доходов: *не относится к высшему классу*
<br>     · Стаж работы: *от 5 до 10 лет*
<br>     · Возраст: *от 30 до 55 лет*
<br>     · Цель кредита: *покупка жилья или проведение свадьбы*
<br>     · Уровень образования: *высшее или среднее*
<br>     · Тип занятости: *госслужащий, либо компаньон*

<br> **Негативно-нейтральный кандидат выглядит так**:
<br>     · Дети: *есть*
<br>     · Семейное положение: *в официальном или гражданском браке, либо никогда не был(а) женат/замужем*
<br>     · Уровень доходов: *не относится к высшему классу*
<br>     · Стаж работы: *от 5 до 10 лет*
<br>     · Возраст: *от 30 до 55 лет*
<br>     · Цель кредита: *хочет получить образование, либо приобрести автомобиль*
<br>     · Уровень образования: *среднее, или неоконченное высшее*
<br>     · Тип занятости: *компаньон, либо наемный работником*

<br> **Картина отрицательного кандидата выглядит следующим образом**:
<br>     · Дети: *от 1 до 2-х*
<br>     · Семейное положение: *в гражданском браке, либо никогда не был(а) женат/замужем*
<br>     · Уровень доходов: *средний*
<br>     · Стаж работы: *менее 5 лет*
<br>     · Возраст: *до 30*
<br>     · Цель кредита: *желает получить образование, либо приобрести автомобиль*
<br>     · Уровень образования: *неоконченное высшее, среднее, либо начальное образование*
<br>     · Тип занятости: *наемный работник*